In [ ]:
# Работа состоит из несольких частей
    - код
    - гипотезы
    - текст
    - обзор литературы

Мы хотим оценить влияние различных адверсариальных атак на современные NLP модели

В рамках данного исследования мы остановимся на задаче классификации

Следовательно: нам нужны
    - Датасеты для классификации
        - 2 на Английском (common domain, specific domain)
        - 2 на Русском (common domain, specific domain)
        
    - Модели для классификации
        - Мы можем использовать TF-IDF на log-reg
        - Bert и его разновидности
            - Английский Берт (Bert-base-uncased, distilled-bert-uncades)
            - Мультиязычный Берт ?
            - Русский Берт (DeepPavlov, дистилированная модель от Давида Деле)
            
    - Атаки:
        - BAE
        - TextFooler
        - Другие атаки из модуля TextAtack

Постановка эксперимента:
    - Пока что не трогаем лог-рег
    - Исследуем Берт
    
Задача1:
    - fine-tuning Берта под задачу классификации
    - Оценка качества на валидации
    - Подготовка адверсариальных примеров на основе валидационного датасета
    - Оценка качества на адверсариальных примерах

Этапы работы:
    1) Загрузка и препроцессинг данных
    2) Fine-tuning соответствующей модели
    3) Валидация
    4) Генерация адверсариальных примеров
    5) Оценка качества
        - Автоматическая валидация:
            - accuracy
            - semantic score
        - Human evaluation
            - Классификация примеров
            - Оценка "реалистичности и грамотности сгенерированных примеров"

## Step 1

In [ ]:
построим пайплайн на основе ноутбука https://www.kaggle.com/kashnitsky/distillbert-catalyst-amazon-product-reviews